# Configuration Interaction with Single Electronic and Photonic Excitations applied to the Pauli-Fierz Hamiltonian

In [1]:
"""Tutorial implementing a CISS-PF program"""

__authors__ = "J. McTague, J. Foley, A. E. DePrince III"
__credits__ = "J. McTague, J. Foley, A. E. DePrince III"
__email__   = "foleyj10@wpunj.edu, deprince@fsu.edu"

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "6/15/2022"




# I. Theoretical Overview

For details on the Pauli-Fierz Hamiltonian and the Hartree-Fock method applied to the Hamiltonian, 
see the [HF-PF tutorial](HF-PF_Tutorial.ipynb).

### *CISS-PF Ansatz*
A mean-field description of the excited states of the molecular system strongly interacting with photonic degrees of freedom, and a correction to the ground-state that contains coupling between the PF-HF reference and simultaneous electronic and photonic excitations, may be obtained through a configuration interaction  ansatz that includes up to simultaneous single excitations of the electronic and photonic degrees of freedom.  Although previously referred to as the CQED-CIS ansatz\cite{Foley_154103}, here we refer to this as the CISS-PF method 
to denote that simultaneous single excitations on the electron and photon degrees of freedom are included in this ansatz, and to denote the specific application 
of this ansatz to the PF Hamiltonian in Eq. (1).   In the following presentation, we assume the PF-HF reference which will be computed by a helper function imported by this notebook.

The polaritonic energy eigenfunctions for state $I$ in the
CISS-PF ansatz can be written as a linear combination of the PF-HF reference and products of all possible single excitations out of the PF-HF reference.  The PF-HF reference involves the product of an electronic Slater determinant with the photon vacuum state $|\Phi_o\rangle |0\rangle$, so single excitations can occur as electronic excitations from an occupied orbital $\phi_i$ to a virtual orbital $\phi_a$, the raising of the photon number state from $|0\rangle \rightarrow |1\rangle$, or both.  We therefore write the CISS-PF wavefunction for state $I$ as
\begin{equation}\label{eq:CISS-PF}
\Psi_I = c_0^0 |\Phi_0\rangle |0\rangle + 
\sum_{i,a} c_{ia}^0 |\Phi_i^a\rangle |0\rangle +
c_0^1 |\Phi_0\rangle |1\rangle +
\sum_{i,a} c_{ia}^1 |\Phi_i^a\rangle |1\rangle \tag{9}. 
\end{equation}
where the coefficients $c$ denote the contribution of a given term to the wavefunction and we have denoted the electronic excitations in the subscript and the photonic excitations in the superscript of these coefficients. For the case of multiple modes, the photonic basis states will be augmented to consider all possible combinations of the occupations of those modes within a maximum photon number.  These coefficients, and the corresponding energy eigenvalues for a given CISS-PF state $I$, may be obtained by diagonalizing the Hamiltonian matrix built in the basis shown below in Eq. (10).  We spin adapt this basis such that $|\Phi_i^a\rangle = \frac{1}{\sqrt{2}}\left(|\Phi_{i\alpha}^{a \alpha} \rangle + |\Phi_{i\beta}^{a\beta}\rangle \right)$, where $\alpha$ and $\beta$ label the spin orbitals as being occupied by spin-up and spin-down electrons, respectively.  The matrix can be written schematically as 
\begin{equation}
\begin{bmatrix}
0 & 0 & 0 & \hbar {\bf g} \\
0 & {\bf A} +\Delta  & \hbar {\bf g}^{\dagger}  & \hbar {\bf G} \\
0 & \hbar {\bf g} & \hbar \omega & 0 \\
\hbar {\bf g}^{\dagger} & \hbar {\bf G} & 0 & {\bf A} + \Delta + \hbar \Omega 
\end{bmatrix}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia}
\end{bmatrix}
=
E_{CISS-PF}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia}, \tag{10}
\end{bmatrix}
\end{equation}
where we have shifted the CISS-PF Hamiltonian by HF-PF energy. 


In the CISS-PF Hamiltonian matrix above, the elements of ${\bf A}$ are similar to Canonical CIS theory,
\begin{equation}
    A_{ia,jb} = (\epsilon_a - \epsilon_i) \delta_{ij} \delta_{ab} + \left( 2(ia|jb) - (ij|ab) \right) \tag{11}, 
\end{equation}
with an important difference being that the two-electron integrals are performed in the HF-PF basis and the orbital energies result from the HF-PF calculation and contain dipole self energy contributions of the reference wavefunction.  The dipole self energy in the subspace of single-excitations is contained in the $\Delta$ matrix,
\begin{equation}
    \Delta_{ia,jb} = 2\sum_{\xi,\xi'} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}_{ia} \mu^{\xi'}_{jb} 
    -\sum_{\xi,\xi'} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}_{ij} \mu^{\xi'}_{ab}\tag{12}. 
\end{equation}
The photonic energy is contained in $\hbar \omega$ and $\hbar \Omega$ as follows:
\begin{equation}
    \Omega_{ia,jb} = \omega \delta_{ij} \delta_{ab}\tag{13}.
\end{equation}
The bilinear coupling contributes to blocks that couple singly-excited determinants to the reference, and to blocks that couple singly-excited determinants in the bra and the ket. 
The coupling between singly-excited determinants and the reference through elements of ${\bf g}$,
\begin{equation}
    g_{ia} = -\sqrt{\omega} \sum_{\xi} \lambda^{\xi} \mu_{ia}^{\xi}\tag{14},
\end{equation}
and the coupling between singly-excited determinants are captured through elements of ${\bf G}$,
\begin{equation}
    G_{ia,jb} = \sqrt{\frac{\omega}{2}} \sum_{\xi} \lambda^{\xi} \left( \mu_{ij}^{\xi} \delta_{ab} 
    -  \mu_{ab}^{\xi} \delta_{ij} \right)\tag{15}.
\end{equation}

## II. Implementation

Using the above overview, we will implement the CISS-PF method using <span style="font-variant: small-caps"> Psi4 </span> and NumPy.  We will also import a helper function that
will perform an intial HF-PF calculation to get the orbital basis for the subsequent CISS-PF calculations.  First, we need to import these Python modules and set some basic Psi4 options. 

In [2]:
# ==> Import Psi4, NumPy, linear algebra package from scipy, time,
#     and helper_cqed_rhf to perform the requisite HF-PF calculation <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import cqed_rhf

# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)



  Memory set to 476.837 MiB by Python driver.


Next we will define the molecular geometry along
with Psi4 options for the basis set and convergence criteria 
for the HF-PF calculation.  The helper function that performs
HF-PF calculations will expect the molecular geometry in a string
representation and options as a dictionary.

In [3]:
# MgH Test Case
#molecule_string = """
#Mg
#H 1 2.2
#symmetry c1
#1 1
#"""

# options dict
#options_dict = {
#    "basis": "cc-pVDZ",
#    "save_jk": True,
#    "scf_type": "pk",
#    "e_convergence": 1e-10,
#    "d_convergence": 1e-10,
#}


molecule_string = """
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
"""


options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8}



Next we will define the parameters that define the photon energy and the coupling
vector. In particular, we will define the photon energy as 

$$ \hbar \omega = 4.75  \: {\rm eV} = 0.1745 \: {\rm a.u.}$$

and the coupling vector as 

$$ \lambda = (0, 0, 0.0125) \: {\rm a.u.} $$

In [4]:
# photon energy
omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV

# coupling vector
#lambda_vector = np.array([0, 0, 0.0125])
lambda_vector = np.array([0, 0, 0.0])


We will run a preliminary RHF calculation using Psi4 and save the wavefunction object 
to help us get 1 and 2-electron integrals.  We will also use the helper_cqed_rhf function to run the HF-PF calculation.

In [5]:
# set the options in psi4
psi4.set_options(options_dict)
# define the molecule based on the string representation of the geometry
mol = psi4.geometry(molecule_string)
# run an RHF calculation with psi4 and save the wavefunction object and RHF energy
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# compute HF-PF wavefunction and energy 
cqed_rhf_dict = cqed_rhf(lambda_vector, molecule_string, options_dict)


Start SCF iterations:

Canonical RHF One-electron energy = -120.1995577633948642
CQED-RHF One-electron energy      = -120.1995577633948642
Nuclear repulsion energy          = 8.0023664821734215
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -74.9420798986807597   dE = -7.49421E+01   dRMS = 4.46835E-12
SCF Iteration   2: Energy = -74.9420798986807597   dE =  0.00000E+00   dRMS = 1.63019E-12
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -74.94207990 hartree
Psi4  SCF energy: -74.94207990 hartree


We need to prepare the arrays that will be used in the elements of the CISS-PF matrix.  We will need access to 2-electron repulsion integrals, dipole integrals, and quadrupole integrals all in the HF-PF basis, and we will also need the orbital energies that arise from HF-PF.  The following block will use results from the previously-run HF-PF calculation on the molecule defined above to obtain the HF-PF orbitals and orbital energies needed to transform all integrals to the HF-PF basis.  These quantities are returned as a dictionary from the HF-PF calculation.  If you want to see all of the available keys, you can do so with the following lines of python:

`for key, items in cqed_rhf_dict.items():
    print(key)`

In [6]:
# grab necessary quantities from cqed_rhf_dict
# molecular RHF energy
scf_e = cqed_rhf_dict["RHF ENERGY"]
# energy from the HF-PF calculation
cqed_scf_e = cqed_rhf_dict["CQED-RHF ENERGY"]
# psi4's wavefunction object
wfn = cqed_rhf_dict["PSI4 WFN"]
# transformation vectors from HF-PF calculation
C = cqed_rhf_dict["CQED-RHF C"]
# CQED-RHF Density matrix
D = cqed_rhf_dict["CQED-RHF DENSITY MATRIX"]
# orbital energies from HF-PF calculation
eps = cqed_rhf_dict["CQED-RHF EPS"]
# dipole moment from HF-PF calculation
cqed_rhf_dipole_moment = cqed_rhf_dict["CQED-RHF DIPOLE MOMENT"]


We will get some more relevant quantities directly from the wavefunction object
that we saved from psi4's RHF calculation.  

In [7]:
# Grab data from wavfunction
# number of doubly occupied orbitals
ndocc = wfn.nalpha()

# total number of orbitals
nmo = wfn.nmo()

# number of virtual orbitals
nvirt = nmo - ndocc



The wavefunction object also has an attribute that contains the transformation vectors that define the AO->MO transformation.  For our CISS-PF calculation, we are interested in the HF-PF transformation vectors rather than the canonical HF transformation vectors. Our HF-PF transformation vectors are now stored to the array `C` from the previous block, and so we can swap these vectors out for the canonical MO vectors by using a set of built-in psi4 functions that maps the wavefunction object to a dictionary, changing the values of the dictionary items that store the canoncal MO transformation vectors to the values that define the HF-PF transformation vectors, and then mapping that dictionary back to a psi4 wavefunction object.  The block below performs these operations.  This seems like a lot of work, but psi4 has some built-in functions that will perform 4-index transformations efficiently utilizing the transformation vectors stored as core Psi4 matrices, so it really benefits us to do this to utilize this machinery!

In [8]:
# need to update the Co and Cv core matrix objects so we can
# utlize psi4s fast integral transformation!
# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# get all orbitals
Ca = wfn.Ca()
# occupied orbitals as psi4 objects but they correspond to CQED-RHF orbitals
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals same way
Cv = wfn.Ca_subset("AO", "VIR")

Now that we have the correct HF-PF transformation vectors (`Co` for the occupied subspace and `Cv` for the virtual subspace), we will obtain the 2-electron repulsion integrals (2ERI) in the HF-PF basis.  We can make use of helper functions associated with the Psi4 mints library to transform the integrals from the AO to HF-PF basis given the transformation vectors as arguments.  We need 2ERIs of the type $(ia|jb)$ and $(ij|ab)$ for the CISS-PF equations, and we will store these integrals in arrays called `ovov` and `oovv`, respectively.

In [9]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())
# 2 electron integrals in CQED-RHF basis

MO_spin = np.asarray(mints.mo_spin_eri(Ca, Ca))

MO_spat = np.asarray(mints.mo_eri(Ca, Ca, Ca, Ca))

# use built-in mints function to build the (ov|ov)
# integrals in the HF-PF basis using Co and Cv
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# use built-in mints function to build the (oo|vv)
# integrals in the HF-PF basis using Co and Cv
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))

Let's see if we can build an equivalent matrix as `MO_spin` using the entries
in `MO_spat` where we have the elements of `MO_spin` given by $\langle ij||kl \rangle$ 
and the elements of `MO_spat` given by $(ij | kl)$

In [42]:
def spin_idx_to_spat_idx_and_spin(P):
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)


def map_spin_to_spat(MO, I, J, K, L):
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = MO[i_s[0], k_s[0], j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = MO[i_s[0], l_s[0], j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk



for i in range(7):
    for j in range(7):
        for k in range(7):
            for l in range(7):
                assert np.isclose(MO_spin[i, j, k, l], map_spin_to_spat(MO_spat, i, j, k, l))
#print(MO_spin[2,1,2,1])
#print(map_spin_to_spat(MO_spat,2,1,2,1))

    

#print(MO_spat[0,0,0,0])
#print(MO_spin[0,1,0,1])

We need the occupied and virtual HF-PF orbital energies, $\epsilon_i$ and $\epsilon_a$ respectively, which we will store in arrays called `\eps_o` and `eps_v`.

In [ ]:
# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]

Terms of the form $\lambda^{\xi} \mu_{pq}^{\xi}$ require dipole integrals in the HF-PF basis dotted into the electric field vector $\vec{\lambda}$. 

Here we will obtain the Cartesian components of the dipole integrals in the AO basis using the mints helper class and store them in arrays `mu_ao_x`, `mu_ao_y`, and `mu_ao_z`.  We will then transform them to the HF-PF basis using the transformation vectors stored in an array called `C`. Finally, we will accumulate the dot product of these integrals and the $\lambda$ vector and store the result in an array called `l_dot_mu_el`.  The terms shown in Eqs. (12), (14), and (15) will utilize the `l_dot_mu_el` array.

In [ ]:
# dipole arrays in AO basis
mu_ao_x = np.asarray(mints.ao_dipole()[0])
mu_ao_y = np.asarray(mints.ao_dipole()[1])
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform dipole array to HF-PF basis
mu_cmo_x = np.dot(C.T, mu_ao_x).dot(C)
mu_cmo_y = np.dot(C.T, mu_ao_y).dot(C)
mu_cmo_z = np.dot(C.T, mu_ao_z).dot(C)

# electronic dipole expectation value with CQED-RHF density
mu_exp_x = np.einsum("pq,pq->", 2 * mu_ao_x, D)
mu_exp_y = np.einsum("pq,pq->", 2 * mu_ao_y, D)
mu_exp_z = np.einsum("pq,pq->", 2 * mu_ao_z, D)

# get electronic dipole expectation value
mu_exp_el = np.array([mu_exp_x, mu_exp_y, mu_exp_z])

# \lambda \cdot < \mu > where < \mu > contains only electronic terms 
l_dot_mu_exp = np.dot(lambda_vector, mu_exp_el)

# \lambda \cdot \mu_{el}
l_dot_mu_el = lambda_vector[0] * mu_cmo_x
l_dot_mu_el += lambda_vector[1] * mu_cmo_y
l_dot_mu_el += lambda_vector[2] * mu_cmo_z

Additionally, we will need the electric field vector dotted into the nuclear dipole moment and the dipole expectation value computed at the HF-PF level.

In [ ]:
# dipole constants to add to CISS-PF Energ
d_c = 0.5 * l_dot_mu_exp**2

# If we have done these transformations correctly, the d_c above
# will agree with what we have from HF-PF calculation
assert np.isclose(d_c, cqed_rhf_dict["DIPOLE ENERGY"])

Build the ${\bf g}$ and ${\bf g}^{\dagger}$ matrices (Eq. (14)). 

In [ ]:
# build g matrix and its adjoint
g = np.zeros((1,ndocc * nvirt), dtype=complex)
g_dag = np.zeros((ndocc * nvirt, 1), dtype=complex)
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia = i * nvirt + a 
        g[0,ia] = (
            -np.sqrt(omega_val) * l_dot_mu_el[i, A]
        )

# Now compute the adjoint of g
g_dag = np.conj(g).T

Build the ${\bf A}$, $\Delta$, $\Omega$, and ${\bf G}$ matrices from Eqs. (11), (12), (13), and (15), respectively. 

In [ ]:
# A
A_matrix = np.zeros((ndocc * nvirt, ndocc * nvirt), dtype=complex)
# Delta
D_matrix = np.zeros((ndocc * nvirt, ndocc * nvirt), dtype=complex)
# G
G = np.zeros((ndocc * nvirt, ndocc * nvirt), dtype=complex)
# \Omega
Omega = np.zeros((ndocc * nvirt, ndocc * nvirt), dtype=complex)

for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia = i * nvirt + a
        
        for j in range(0, ndocc):
            for b in range(0, nvirt):
                B = b + ndocc
                jb = j * nvirt + b
                
                # ERI contribution to A + \Delta
                A_matrix[ia, jb] = (2.0 * ovov[i, a, j, b] - oovv[i, j, a, b])
                
                # 2-electron dipole contribution to A + \Delta
                D_matrix[ia, jb] += 2.0 * l_dot_mu_el[i, A] * l_dot_mu_el[j, B]
                D_matrix[ia, jb] -= l_dot_mu_el[i, j] * l_dot_mu_el[A, B]
                
                # bilinear coupling contributions to G
                # off-diagonal terms (plus occasional diagonal terms)
                G[ia, jb] += np.sqrt(omega_val / 2) * l_dot_mu_el[i, j] * (a == b)
                G[ia, jb] -= np.sqrt(omega_val / 2) * l_dot_mu_el[A, B] * (i == j)
                
                # diagonal contributions to A_p_D, G, and \Omega matrix
                if i == j and a == b:
                    # orbital energy contribution to A + \Delta ... this also includes 
                    # the DSE terms that contributed to the CQED-RHF energy 
                    A_matrix[ia, jb] += eps_v[a] 
                    A_matrix[ia, jb] -= eps_o[i] 
                    
                    # diagonal \omega term
                    Omega[ia, jb] = omega_val

Now that we have the sub-blocks formed, we can map these subblocks to the supermatrix that comprises the
CISS-PF matrix.  The ordering of the basis vectors can be illustrated schematically as follows:

\begin{equation}
|R,0\rangle, |S,0\rangle, |R,1\rangle, |S,1\rangle
\end{equation}
where $|R\rangle$ denotes the reference electronic basis state (of which there is only 1) and $|S\rangle$ denotes all singly-excited electronic basis states (of which there are $N_v N_o$ states).  Therefore, we can define 
offsets for the blocks of the matrices as follows:

$|R,0\rangle$ offset = $0$

$|S,0\rangle$ offset = $1$

$|R,1\rangle$ offset = $N_v N_o + 1$

$|S,1\rangle$ offset = $N_v N_o + 2$

Using the same basis states but neglecting the $\Delta$ matrix from the CISS-PF method, we arrive at a CISS approach to the Jaynes-Cummings Hamiltonian:
\begin{equation}\label{eq:H-CISS-JC}
\begin{bmatrix}
0 & 0 & 0 & \hbar {\bf g} \\
0 & {\bf A}  & \hbar {\bf g}^{\dagger}  & \hbar {\bf G} \\
0 & \hbar {\bf g} & \hbar \omega & 0 \\
\hbar {\bf g}^{\dagger} & \hbar {\bf G} & 0 & {\bf A} + \hbar \Omega 
\end{bmatrix}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia}
\end{bmatrix}
=
E_{CISS-JC}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia}, \tag{16}
\end{bmatrix}
\end{equation}

In [ ]:
# define the offsets
R0_offset = 0
S0_offset = 1
R1_offset = ndocc * nvirt + 1
S1_offset = ndocc * nvirt + 2

In [ ]:
# CISS Hamiltonians
H_CISS_PF = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H_CISS_JC = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)

# build the supermatrix
# g coupling
# PF
H_CISS_PF[R0_offset:S0_offset, S1_offset:] = g
H_CISS_PF[S0_offset:R1_offset, R1_offset:S1_offset] = g_dag
H_CISS_PF[R1_offset:S1_offset, S0_offset:R1_offset] = g
H_CISS_PF[S1_offset:,          R0_offset:S0_offset] = g_dag
# JC
H_CISS_JC[R0_offset:S0_offset, S1_offset:] = g
H_CISS_JC[S0_offset:R1_offset, R1_offset:S1_offset] = g_dag
H_CISS_JC[R1_offset:S1_offset, S0_offset:R1_offset] = g
H_CISS_JC[S1_offset:,          R0_offset:S0_offset] = g_dag

# A + \Delta for PF
H_CISS_PF[S0_offset:R1_offset, S0_offset:R1_offset] = A_matrix + D_matrix

# A for JC
H_CISS_JC[S0_offset:R1_offset, S0_offset:R1_offset] = A_matrix

# omega
# PF
H_CISS_PF[R1_offset, R1_offset] = omega_val
# JC
H_CISS_JC[R1_offset, R1_offset] = omega_val

# A + \Delta + \Omega for PF
H_CISS_PF[S1_offset:, S1_offset:] = A_matrix + D_matrix + Omega

# A + \Omega for JC
H_CISS_JC[S1_offset:, S1_offset:] = A_matrix + Omega

# G coupling
# PF
H_CISS_PF[S1_offset:,S0_offset:R1_offset] = G
H_CISS_PF[S0_offset:R1_offset, S1_offset:] = G
# JC
H_CISS_JC[S1_offset:,S0_offset:R1_offset] = G
H_CISS_JC[S0_offset:R1_offset, S1_offset:] = G

If we eliminate the simultaneous single-excitations in the photonic and electronic
degrees of freedom, we arrive at the CIS-PF method which has direct analogy to the TDA-PF
method described by Shao and co-workers:
\begin{equation}
\begin{bmatrix}
{\bf A} +\Delta  & \hbar {\bf g}^{\dagger}  \\
 \hbar {\bf g} & \hbar \omega
\end{bmatrix}
\begin{bmatrix}
{\bf c}^0_{ia} \\
{\bf c}^1_0 
\end{bmatrix}
=
E_{CIS-PF}
\begin{bmatrix}
{\bf c}^0_{ia} \\
{\bf c}^1_0
\end{bmatrix}. \tag{17}
\end{equation}

If we neglect the $\Delta$ matrix in the CIS-PF Hamiltonian, we arrive at the CIS-JC method in direct analogy with the TDA-JC method described by Shao and co-workers:
\begin{equation}
\begin{bmatrix}
{\bf A}  & \hbar {\bf g}^{\dagger}  \\
 \hbar {\bf g} & \hbar \omega
\end{bmatrix}
\begin{bmatrix}
{\bf c}^0_{ia} \\
{\bf c}^1_0 
\end{bmatrix}
=
E_{CIS-JC}
\begin{bmatrix}
{\bf c}^0_{ia} \\
{\bf c}^1_0
\end{bmatrix}. \tag{18}
\end{equation}


The ordering of the basis vectors for the two CIS methods above can be illustrated schematically as follows:

\begin{equation}
|S,0\rangle, |R,1\rangle
\end{equation}
We can define 
offsets for the blocks of the matrices as follows:

$|S,0\rangle$ offset = $0$

$|R,1\rangle$ offset = $N_v N_o$

In [ ]:
# define the CIS offsets
CIS_S0_offset = 0
CIS_R1_offset = ndocc * nvirt

# CIS Hamiltonians
H_CIS_PF = np.zeros((ndocc * nvirt + 1, ndocc * nvirt + 1), dtype=complex)
H_CIS_JC = np.zeros((ndocc * nvirt + 1, ndocc * nvirt + 1), dtype=complex)

# build the supermatrix
# g coupling
# PF
H_CIS_PF[CIS_R1_offset:, CIS_S0_offset:CIS_R1_offset] = g
H_CIS_PF[CIS_S0_offset:CIS_R1_offset, CIS_R1_offset:] = g_dag
# JC
H_CIS_JC[CIS_R1_offset:, CIS_S0_offset:CIS_R1_offset] = g
H_CIS_JC[CIS_S0_offset:CIS_R1_offset, CIS_R1_offset:] = g_dag

# A + \Delta for PF
H_CIS_PF[CIS_S0_offset:CIS_R1_offset, CIS_S0_offset:CIS_R1_offset] = A_matrix + D_matrix
# A  for JF
H_CIS_JC[CIS_S0_offset:CIS_R1_offset, CIS_S0_offset:CIS_R1_offset] = A_matrix

# omega
# PF
H_CIS_PF[CIS_R1_offset, CIS_R1_offset] = omega_val
# JC
H_CIS_JC[CIS_R1_offset, CIS_R1_offset] = omega_val

# diagonalize the total QED-CIS matrix and the 
E_CISS_PF, C_CISS_PF = np.linalg.eigh(H_CISS_PF)
E_CISS_JC, C_CISS_JC = np.linalg.eigh(H_CISS_JC)

E_CIS_PF, C_CIS_PF = np.linalg.eigh(H_CIS_PF)
E_CIS_JC, C_CIS_JC = np.linalg.eigh(H_CIS_JC)



Now we can diagonalize the different supermatrices formed above to obtain the
eigenvalues and eigenvectors from these 4 related levels of theory.

In [ ]:
# diagonalize the 4 different supermatrices formed above
E_CISS_PF, C_CISS_PF = np.linalg.eigh(H_CISS_PF)
E_CISS_JC, C_CISS_JC = np.linalg.eigh(H_CISS_JC)

E_CIS_PF, C_CIS_PF = np.linalg.eigh(H_CIS_PF)
E_CIS_JC, C_CIS_JC = np.linalg.eigh(H_CIS_JC)

# store the results in a dictionary
cqed_cis_dict = {
    "RHF ENERGY": scf_e,
    "CQED-RHF ENERGY": cqed_scf_e,
    "CISS-PF ENERGY": E_CISS_PF,
    "CISS-JC ENERGY": E_CISS_JC,
    "CIS-PF ENERGY": E_CIS_PF,
    "CIS-JC ENERGY": E_CIS_JC,
}


#### NEED SOME VALIDATION CHECKS HERE!

In [ ]:
expected_vals = np.array([-2.50831265e-04,  
                          1.65570838e-01,
                          1.82666685e-01,
                          2.33013344e-01,
                          2.33013344e-01])


assert np.allclose(expected_vals, cqed_cis_dict['CISS-PF ENERGY'][0:5],1e-6)

### References

   - [[McTague:2022:154103]](https://aip.scitation.org/doi/10.1063/5.0091953)] J. McTague, J. J. Foley IV, *J. Chem. Phys.* **156**, 154103 (2022)
   - [[Shao:2021:064107]](https://aip.scitation.org/doi/full/10.1063/5.0057542) J. Yang, Q. Ou, Z. Pei, H. Wang, B. Weng, Z. Shuai, K. Mullen, Y. Shao, *J. Chem. Phys.* **155**, 064107 (2021)